In [1]:
!pip install robobrowser chardet

「chardet」は、ファイルやページの文字コードを判定するためのライブラリ。「CHARacter DETection」の略

自動操作
1. Googleのトップページを開く
2. 検索語を入力する
3. Googleの検索ボタンを押す
4. 検索結果がm表示される

ステートレスなクローラーであれば、検索結果のURLが
https://www.google.co.jp/search?q=Python 
になるというように、URLを組み立ててページを取得する
    
RoboBrowserではURLを意識せず通常のブラウザを操作するのと似た感覚でプログラムを書ける    

In [2]:
from robobrowser import RoboBrowser

# RoboBrowserオブジェクトを作成する。キーワード引数parserはBeautifulSoup()の第2引数として使われる。
browser = RoboBrowser(parser='html.parser')

browser.open('https://www.google.co.jp/')  # open()メソッドでGoogleのトップページを開く。

# 検索語を入力して送信する。
form = browser.get_form(action='/search')  # フォームを取得。
form['q'] = 'Python'  # フォームのqという名前のフィールドに検索語を入力。
browser.submit_form(form, list(form.submit_fields.values())[0])  # 一つ目のボタン（Google 検索）を押す。

# 検索結果のタイトルとURLを抽出して表示する。
# select()メソッドはBeautiful Soupのselect()メソッドと同じものであり、
# 引数のCSSセレクターにマッチする要素に対応するTagオブジェクトのリストを取得できる。
for a in browser.select('h3 > a'):
    print(a.text)
    print(a.get('href'))

Python - ウィキペディア
/url?q=https://ja.wikipedia.org/wiki/Python&sa=U&ved=0ahUKEwi-mvqqwdbXAhXMu7wKHc6xDFIQFggUMAA&usg=AOvVaw1nTNqDrah-f21E_ZGVE4iP
Pythonとは - python.jp
/url?q=https://www.python.jp/about/&sa=U&ved=0ahUKEwi-mvqqwdbXAhXMu7wKHc6xDFIQFggfMAE&usg=AOvVaw3wOtylaAtTWC6pYGfoQclG
専門知識いらず！Pythonとは？言語の特徴から網羅的に徹底解説 | 侍 ...
/url?q=https://www.sejuku.net/blog/7720&sa=U&ved=0ahUKEwi-mvqqwdbXAhXMu7wKHc6xDFIQFgglMAI&usg=AOvVaw3PXNMs2wZvWcLj2zld_W4k
Python基礎講座(1 Pythonとは) - Qiita
/url?q=https://qiita.com/Usek/items/ff4d87745dfc5d9b85a4&sa=U&ved=0ahUKEwi-mvqqwdbXAhXMu7wKHc6xDFIQFggvMAM&usg=AOvVaw35Hsogh0B7zKrA-g8qostK
Python 13299 posts - Qiita
/url?q=https://qiita.com/tags/Python&sa=U&ved=0ahUKEwi-mvqqwdbXAhXMu7wKHc6xDFIQFgg2MAQ&usg=AOvVaw1yBDWlBfqJf56ETHHHavBN
Python 3を使うべきでない場合（なんてない） | プログラミング | POSTD
/url?q=http://postd.cc/case-python-3/&sa=U&ved=0ahUKEwi-mvqqwdbXAhXMu7wKHc6xDFIQFgg7MAU&usg=AOvVaw3E3uFbpj2aor2TiU9F7G_S
Pythonとは？何に使えるの？Pythonの特徴や使い道を…｜Udemy ...
/url?q=https://udemy.beness

※上記のサンプルはRoboBrowserの使い方を示すためのもの。
   Googleの検索結果を高頻度でクロールすると、CAPTCHAと呼ばれる画像が表示される。

# ログインが必要なWebサイトからデータを収集

事前にパスワードを入力した .envファイルを作成する

AMAZON_EMAIL=

AMAZON_PASSWORD=

In [ ]:
import sys
import os

from robobrowser import RoboBrowser

# 認証の情報は環境変数から取得する。
AMAZON_EMAIL = os.environ['AMAZON_EMAIL']
AMAZON_PASSWORD = os.environ['AMAZON_PASSWORD']

# RoboBrowserオブジェクトを作成する。
browser = RoboBrowser(
    parser='html.parser',  # Beautiful Soupで使用するパーサーを指定する。
    # Cookieが使用できないと表示されてログインできない問題を回避するため、
    # 通常のブラウザーのUser-Agent（ここではFirefoxのもの）を使う。
    user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:45.0) Gecko/20100101 Firefox/45.0')


def main():
    # 注文履歴のページを開く。
    print('Navigating...', file=sys.stderr)
    browser.open('https://www.amazon.co.jp/gp/css/order-history')

    # サインインページにリダイレクトされていることを確認する。
    assert 'Amazonサインイン' in browser.parsed.title.string

    # name="signIn" というサインインフォームを埋める。
    # フォームのname属性の値はブラウザーの開発者ツールで確認できる。
    form = browser.get_form(attrs={'name': 'signIn'})
    form['email'] = AMAZON_EMAIL  # name="email" という入力ボックスを埋める。
    form['password'] = AMAZON_PASSWORD  # name="password" という入力ボックスを埋める。

    # フォームを送信する。正常にログインするにはRefererヘッダーとAccept-Languageヘッダーが必要。
    print('Signing in...', file=sys.stderr)
    browser.submit_form(form, headers={
        'Referer': browser.url,
        'Accept-Language': 'ja,en-US;q=0.7,en;q=0.3',
    })

    # ログインに失敗する場合は、次の行のコメントを外してHTMLのソースを確認すると良い。
    # print(browser.parsed.prettify())

    # ページャーをたどる。
    while True:
        assert '注文履歴' in browser.parsed.title.string  # 注文履歴画面が表示されていることを確認する。

        print_order_history()  # 注文履歴を表示する。

        link_to_next = browser.get_link('次へ')  # 「次へ」というテキストを持つリンクを取得する。
        if not link_to_next:
            break  # 「次へ」のリンクがない場合はループを抜けて終了する。

        print('Following link to next page...', file=sys.stderr)
        browser.follow_link(link_to_next)  # 「次へ」というリンクをたどる。


def print_order_history():
    """
    現在のページのすべての注文履歴を表示する。
    """
    # ページ内のすべての注文履歴について反復する。ブラウザーの開発者ツールでclass属性の値を確認できる。
    for line_item in browser.select('.order-info'):
        order = {}  # 注文の情報を格納するためのdict。
        # 注文の情報のすべての列について反復する。
        for column in line_item.select('.a-column'):
            label_element = column.select_one('.label')
            value_element = column.select_one('.value')
            # ラベルと値がない列は無視する。
            if label_element and value_element:
                label = label_element.get_text().strip()
                value = value_element.get_text().strip()
                order[label] = value  # 注文の情報を格納する。

        print(order['注文日'], order['合計'])  # 注文の情報を表示する。

if __name__ == '__main__':
    main()

In [3]:
!forego run python amazon_order_history.py

Navigating...
Signing in...
2017年11月21日 ￥ 0
2017年11月6日 ￥ 374
2017年11月3日 ￥ 0
2017年11月3日 ￥ 0
2017年10月6日 ￥ 0
2017年10月4日 ￥ 0
2017年9月30日 ￥ 0
2017年9月30日 ￥ 0
2017年9月4日 ￥ 0
2017年8月4日 ￥ 0
Following link to next page...
2017年8月1日 ￥ 0
2017年7月27日 ￥ 0
2017年7月19日 ￥ 0
2017年7月11日 ￥ 0
2017年7月11日 ￥ 0
2017年7月10日 ￥ 0
2017年7月9日 ￥ 0
2017年7月8日 ￥ 0
2017年6月25日 ￥ 0
2017年6月24日 ￥ 0
Following link to next page...
2017年6月24日 ￥ 0
2017年6月24日 ￥ 0
2017年6月5日 ￥ 0
2017年5月29日 ￥ 0
